In [70]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load embeddings
clap_embeddings = np.load("./audio_embeddings_clap.npy", allow_pickle=True).item()
mert_embeddings = np.load("./audio_embeddings_mert.npy", allow_pickle=True).item()

# Load Cocola scores from CSV
cocola_scores_df = pd.read_csv("./cocola_scores.csv")

# Colonnes de cocola
cocola_columns = cocola_scores_df.columns
print(cocola_columns)



Index(['vocals', 'instrumental', 'score'], dtype='object')


In [71]:
embedding_matrix_clap = np.load("./audio_embeddings_clap.npy", allow_pickle=True)
print(type(embedding_matrix_clap))
embedding_matrix_clap = embedding_matrix_clap.item()
print(type(embedding_matrix_clap))
file_names = list(embedding_matrix_clap.keys())


# Récupère les vecteurs d'embedding (valeurs du dictionnaire)
embeddings_list_clap = list(embedding_matrix_clap.values())
embedding_matrix_clap = np.vstack(embeddings_list_clap)  # Combine les vecteurs en une matrice
print(f"Nouvelle forme de la matrice d'embeddings : {embedding_matrix_clap.shape}")

<class 'numpy.ndarray'>
<class 'dict'>
Nouvelle forme de la matrice d'embeddings : (42, 512)


In [72]:
embedding_matrix_mert = np.load("./audio_embeddings_mert.npy", allow_pickle=True)
print(type(embedding_matrix_mert))
embedding_matrix_mert = embedding_matrix_mert.item()
print(type(embedding_matrix_mert))
file_names = list(embedding_matrix_mert.keys())


# Récupère les vecteurs d'embedding (valeurs du dictionnaire)
embeddings_list_mert = list(embedding_matrix_mert.values())
embedding_matrix_mert = np.vstack(embeddings_list_mert)  # Combine les vecteurs en une matrice
print(f"Nouvelle forme de la matrice d'embeddings : {embedding_matrix_mert.shape}")

<class 'numpy.ndarray'>
<class 'dict'>
Nouvelle forme de la matrice d'embeddings : (42, 768)


In [73]:
# Calculer la matrice de similarité cosinus
cosine_sim_matrixc = cosine_similarity(embedding_matrix_clap)

# Afficher la forme de la matrice (N, N) et un aperçu
print(f"Forme de la matrice de similarité cosinus : {cosine_sim_matrixc.shape}")
print(cosine_sim_matrixc[:5, :5])  # Afficher les 5 premières lignes et colonnes



Forme de la matrice de similarité cosinus : (42, 42)
[[1.0000002  0.34097084 0.83422333 0.5616863  0.65733886]
 [0.34097084 1.0000002  0.16409242 0.5731591  0.15953967]
 [0.83422333 0.16409242 0.99999994 0.4414206  0.6725746 ]
 [0.5616863  0.5731591  0.4414206  1.0000004  0.38131076]
 [0.65733886 0.15953967 0.6725746  0.38131076 0.9999998 ]]


In [74]:
# Calculer la matrice de similarité cosinus
cosine_sim_matrixm = cosine_similarity(embedding_matrix_mert)

# Afficher la forme de la matrice (N, N) et un aperçu
print(f"Forme de la matrice de similarité cosinus : {cosine_sim_matrixm.shape}")
print(cosine_sim_matrixm[:5, :5])  # Afficher les 5 premières lignes et colonnes

Forme de la matrice de similarité cosinus : (42, 42)
[[ 0.99999994 -0.92627406  0.66491747 -0.76870966  0.03508142]
 [-0.92627406  1.         -0.7857152   0.83416104  0.14618175]
 [ 0.66491747 -0.7857152   1.0000001  -0.54862237 -0.5991662 ]
 [-0.76870966  0.83416104 -0.54862237  1.          0.02676624]
 [ 0.03508142  0.14618175 -0.5991662   0.02676624  0.9999999 ]]


In [75]:
# On suppose que `cocola` est un DataFrame avec les colonnes ['vocals', 'instrumental', 'score']
import pandas as pd
import numpy as np

# Supposons que `cosine_sim_matrixc` est une matrice numpy (42x42 par exemple)

# Extraction des valeurs de similarité pour vocals (21 premières lignes et colonnes)
vocals_similarity = cosine_sim_matrixc[21:42, 21:42]  # 21x21

# Ajout des valeurs de similarité dans le DataFrame cocola
similarity_values = []
for i in range(21):  # Pour chaque vocal (21 premières lignes)
    for j in range(21):  # Correspondant aux instrumentals (21 suivantes)
        similarity_values.append(vocals_similarity[i, j])

# Ajout de la nouvelle colonne
cocola_scores_df["CLAP"] = similarity_values




In [76]:
print(cocola_scores_df.head())

           vocals                                      instrumental  \
0  Bam Bam - Hi-Q                                    Bam Bam - Hi-Q   
1  Bam Bam - Hi-Q  Benji Cossa - New Flowers (Fast 4-track Version)   
2  Bam Bam - Hi-Q               Bessie Smith - My Sweetie Went Away   
3  Bam Bam - Hi-Q             David Rovics - We Just Want The World   
4  Bam Bam - Hi-Q                   deef - Ein sonniger Tag mit dir   

       score      CLAP  
0  45.754675  1.000000  
1  46.145325  0.255522  
2  44.335644  0.630813  
3  46.341496  0.289742  
4  43.764599  0.459481  


In [77]:
#null values
print(cocola_scores_df.isnull().sum())

vocals          0
instrumental    0
score           0
CLAP            0
dtype: int64


In [87]:
# Normalisation et centrage des valeurs de score
cocola_scores_df["normalized_score"] = (cocola_scores_df["score"] - cocola_scores_df["score"].mean()) / cocola_scores_df["score"].std()
cocola_scores_df["normalized_CLAP"] = (cocola_scores_df["CLAP"] - cocola_scores_df["CLAP"].mean()) / cocola_scores_df["CLAP"].std()

# Calcul de la corrélation entre les scores normalisés et les valeurs de similarité
correlation = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_CLAP"])
print(f"Corrélation entre les scores et les valeurs de similarité : {correlation}")
#calcul de la corrélation de Spearman
correlation_spearman = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_CLAP"], method="spearman")
print(f"Corrélation de Spearman entre les scores et les valeurs de similarité : {correlation_spearman}")
#calcul de la corrélation de Kendall
correlation_kendall = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_CLAP"], method="kendall")
print(f"Corrélation de Kendall entre les scores et les valeurs de similarité : {correlation_kendall}")

Corrélation entre les scores et les valeurs de similarité : 0.05109174349230319
Corrélation de Spearman entre les scores et les valeurs de similarité : 0.07905827255516565
Corrélation de Kendall entre les scores et les valeurs de similarité : 0.05312077256961182


In [79]:
# Même chose pour MERT mais [:21, 21:42]
# Extraction des valeurs de similarité pour vocals (21 premières lignes et colonnes)
vocals_similaritym = cosine_sim_matrixm[:21, 21:42]  # 21x21

mert_values = []
for i in range(21):  # Pour chaque vocal (21 premières lignes)
    for j in range(21):  # Correspondant aux instrumentals (21 suivantes)
        mert_values.append(vocals_similaritym[i, j])

# Ajout de la nouvelle colonne
cocola_scores_df["MERT"] = mert_values
# Normalisation et centrage des valeurs de score
cocola_scores_df["normalized_MERT"] = (cocola_scores_df["MERT"] - cocola_scores_df["MERT"].mean()) / cocola_scores_df["MERT"].std()

In [81]:
# Calcul de la corrélation entre les scores normalisés et les valeurs de similarité
correlation = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_MERT"])
print(f"Corrélation entre les scores et les valeurs de similarité : {correlation}")

# Calcul de la corrélation de Spearman
spearman_correlation = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_MERT"], method="spearman")
print(f"Corrélation de Spearman entre les scores et les valeurs de similarité : {spearman_correlation}")

# Calcul de la corrélation de kendall
kendall_correlation = cocola_scores_df["normalized_score"].corr(cocola_scores_df["normalized_MERT"], method="kendall")
print(f"Corrélation de Kendall entre les scores et les valeurs de similarité : {kendall_correlation}")

Corrélation entre les scores et les valeurs de similarité : -0.018302057090385595
Corrélation de Spearman entre les scores et les valeurs de similarité : -0.016549836717903946
Corrélation de Kendall entre les scores et les valeurs de similarité : -0.010327767470624612


In [82]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Définition des variables indépendantes et dépendantes
X = cocola_scores_df["score"].values.reshape(-1, 1)  # X doit être en 2D
y = cocola_scores_df["CLAP"].values

# Régression linéaire
model = LinearRegression()
model.fit(X, y)

# Afficher les résultats
print(f"Coefficient de régression : {model.coef_}")
print(f"Ordonnée à l'origine : {model.intercept_}")
print(f"R² (coefficient de détermination) : {model.score(X, y)}")



Coefficient de régression : [0.0049813]
Ordonnée à l'origine : 0.09516064020165937
R² (coefficient de détermination) : 0.002610366414126264


In [83]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Définition des variables indépendantes et dépendantes
X = cocola_scores_df["normalized_score"].values.reshape(-1, 1)  # X doit être en 2D
y = cocola_scores_df["MERT"].values

# Régression linéaire
model = LinearRegression()
model.fit(X, y)

# Afficher les résultats
print(f"Coefficient de régression : {model.coef_}")
print(f"Ordonnée à l'origine : {model.intercept_}")
print(f"R² (coefficient de détermination) : {model.score(X, y)}")



Coefficient de régression : [-0.01047648]
Ordonnée à l'origine : -0.0036104229672720978
R² (coefficient de détermination) : 0.0003349653416386822


In [84]:
# afficher top 10 des scores
top10 = cocola_scores_df.sort_values(by="score", ascending=False).head(10)
print(top10)

                                 vocals  \
45  Bessie Smith - My Sweetie Went Away   
56  Bessie Smith - My Sweetie Went Away   
51  Bessie Smith - My Sweetie Went Away   
43  Bessie Smith - My Sweetie Went Away   
61  Bessie Smith - My Sweetie Went Away   
52  Bessie Smith - My Sweetie Went Away   
50  Bessie Smith - My Sweetie Went Away   
53  Bessie Smith - My Sweetie Went Away   
42  Bessie Smith - My Sweetie Went Away   
54  Bessie Smith - My Sweetie Went Away   

                                        instrumental      score      CLAP  \
45             David Rovics - We Just Want The World  58.303272  0.133987   
56                    Short Hand - Certain Strangers  57.926910  0.470327   
51                         Los Steaks - Sunday Girls  57.922951  0.218267   
43  Benji Cossa - New Flowers (Fast 4-track Version)  57.745975  0.219688   
61                             Wann - Happy Birthday  57.739521  0.238412   
52                 Mia Doi Todd - I gave you my home  57.255163 

In [85]:
# sauver le csv
cocola_scores_df.to_csv("mesures_mashups.csv", index=False)

In [86]:
# top 10 et bottom 10 des scores
top10 = cocola_scores_df.sort_values(by="score", ascending=False).head(10)
bottom10 = cocola_scores_df.sort_values(by="score", ascending=True).head(10)

# Afficher les résultats
print("Top 10 des scores :")
print(top10)
print("\nBottom 10 des scores :")
print(bottom10)

Top 10 des scores :
                                 vocals  \
45  Bessie Smith - My Sweetie Went Away   
56  Bessie Smith - My Sweetie Went Away   
51  Bessie Smith - My Sweetie Went Away   
43  Bessie Smith - My Sweetie Went Away   
61  Bessie Smith - My Sweetie Went Away   
52  Bessie Smith - My Sweetie Went Away   
50  Bessie Smith - My Sweetie Went Away   
53  Bessie Smith - My Sweetie Went Away   
42  Bessie Smith - My Sweetie Went Away   
54  Bessie Smith - My Sweetie Went Away   

                                        instrumental      score      CLAP  \
45             David Rovics - We Just Want The World  58.303272  0.133987   
56                    Short Hand - Certain Strangers  57.926910  0.470327   
51                         Los Steaks - Sunday Girls  57.922951  0.218267   
43  Benji Cossa - New Flowers (Fast 4-track Version)  57.745975  0.219688   
61                             Wann - Happy Birthday  57.739521  0.238412   
52                 Mia Doi Todd - I gave you